In [1]:
%load_ext autoreload
%autoreload 2
from models.bilstm_crf import BiLstm_CRF
import torch
import torch.optim as optim
import utils.solver as solver
from models.bilstm_crf import BiLstm_CRF
import utils.data_reader as data_reader



In [4]:
torch.manual_seed(1)
torch.set_num_threads(torch.get_num_threads() // 3)
device = torch.device('cuda')

START_TAG = "<START>"
STOP_TAG = "<END>"
EMBEDDING_DIM = 128
HIDDEN_DIM = 128

# Make up some training data
(word2idx, idx2word), (tag2idx, idx2tag) = data_reader.read_word_and_tag(
        './mid/vocab.txt', './data/lab.txt')
seqs_train, tags_train, intents_train = data_reader.read_seqtag_data_with_unali_act(
        './data/train.txt', word2idx, tag2idx)
training_data = list(zip(seqs_train, tags_train))
# print(training_data)
model = BiLstm_CRF(word2idx, tag2idx, EMBEDDING_DIM, HIDDEN_DIM, 1, device=device).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

print(training_data[0][1])
# Check predictions before training
with torch.no_grad():
    precheck_sent = training_data[0][0]
    precheck_tags = training_data[0][1]
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(1):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        targets = torch.tensor(tags, dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.loss(sentence, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = training_data[0][0]
    print(model(precheck_sent))
# We got it!

Reading source data ...
[128, 128, 128, 128, 128, 4, 128, 45, 128, 50, 111, 128, 80, 125, 125]
([95, 34, 84, 49, 21, 24, 26, 72, 108, 32, 80, 95, 34, 84, 49], tensor(47.0206, device='cuda:0'))
([128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 80], tensor(53.8885, device='cuda:0'))


In [ ]:
a = [1,2,3]
b = [5,6,7]
list(zip(a,b))